In [1]:
import numpy as np

In [2]:
def american_option_binomial_tree(S, K, r, sigma, T, N, option_type = 'C'):
    """
    Price an American call option using a binomial tree.
    
    S : float : initial stock price
    K : float : strike price
    r : float : risk-free interest rate
    sigma : float : volatility
    T : float : time to maturity (in years)
    N : int : number of time steps
    option_type: str : the type of option, default Call
    """
    # Calculate the parameters of the binomial tree model
    delta_t = T / N
    # up-move factor
    u = np.exp(sigma * np.sqrt(delta_t))
    #down-move factor
    d = 1 / u
    # probability of up-move
    p = (1 + r - d) / (u - d)
    # probability of down-move
    q = 1 - p
    
    # Initialize the stock price and option value arrays
    stock_price = np.zeros((N + 1, N + 1))
    option_value = np.zeros((N + 1, N + 1))
    
    if option_type == 'C':
        # Calculate stock prices at each time step
        for i in range(N + 1):
            for j in range(i + 1):
                stock_price[j, i] = S * (u ** (i - j)) * (d ** j)

        # Calculate option value at each time step
        for j in range(N + 1):
            option_value[j, N] = max(0, stock_price[j, N] - K)
            
        # Calculate the option value at earlier times using backward induction
        for i in range(N - 1, -1, -1):
            for j in range(i + 1):
                exercise = (stock_price[j, i] - K)
                hold = np.exp(-r * delta_t) * (p * option_value[j, i + 1] + q * option_value[j + 1, i + 1])
                option_value[j, i] = max(exercise, hold)
         # Return the option price at time 0
        return option_value[0, 0]
    
    else:
        for i in range(N+1):
            for j in range(i+1):
                stock_price[j, i] = S * (u ** (i-j)) * (d ** j)
                option_value[j, i] = max(K - stock_price[j, i], 0)


        for i in range(N-1, -1, -1):
            for j in range(i+1):
                exercise_value = K - stock_price[j, i]
                expected_value = p * option_value[j, i+1] + q * option_value[j+1, i+1]
                option_value[j, i] = max(exercise_value, expected_value)

       
        return option_value[0, 0]

# Example usage
S = 60
K = 55
r = 0.05
sigma = 0.7
T = 1
N = 4

american_call_price = american_option_binomial_tree(S, K, r, sigma, T, N, 'P')
print('American put option price:', american_call_price)

American put option price: 10.087176434931456
